In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [1]:
import tensorflow as tf
import sklearn.model_selection as sk
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [7]:
# ds_path = "/content/drive/MyDrive/Datasets/products.csv"
ds_path = "./dataset/products.csv"
model_save_path = "./model"
EPOCHS = 10
VOCAB_SIZE = 2000
BATCH_SIZE = 64
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 64

In [3]:
df = pd.read_csv(ds_path)
df.category = df.category.map(lambda x: x.split("|")[0].replace("-", " "))
labels = list(pd.get_dummies(df.category))
label_count = len(labels)
print("Label count:", label_count)
titles = df.title.values.tolist()

Label count: 1189


In [4]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="[OOV]")
tokenizer.fit_on_texts(titles)
X = pad_sequences(tokenizer.texts_to_sequences(titles), maxlen=MAX_SEQUENCE_LENGTH)
print("Shape of data tensor:", X.shape)
Y = pd.get_dummies(df.category).values
print("Shape of label tensor:", Y.shape)
X_train, X_test, Y_train, Y_test = sk.train_test_split(
    X, Y, test_size=0.2, random_state=1
)

Shape of data tensor: (99972, 50)
Shape of label tensor: (99972, 1189)


In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(
            input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=X.shape[1]
        ),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(EMBEDDING_DIM, dropout=0.2, recurrent_dropout=0.2)
        ),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(label_count, activation="softmax"),
    ]
)

In [ ]:
print("Compiling the model...")
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)
print("Compiled")

Compiling the model...
Compiled


In [10]:
history = model.fit(
    X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2
)

Epoch 1/10
1000/1000 [==============================] - 222s 216ms/step - loss: 4.7086 - accuracy: 0.1506 - val_loss: 3.2027 - val_accuracy: 0.3774
Epoch 2/10
1000/1000 [==============================] - 214s 214ms/step - loss: 2.4573 - accuracy: 0.5199 - val_loss: 1.7815 - val_accuracy: 0.6584
Epoch 3/10
1000/1000 [==============================] - 218s 218ms/step - loss: 1.5633 - accuracy: 0.6877 - val_loss: 1.2871 - val_accuracy: 0.7574
Epoch 4/10
1000/1000 [==============================] - 213s 213ms/step - loss: 1.1923 - accuracy: 0.7584 - val_loss: 1.0762 - val_accuracy: 0.8018
Epoch 5/10
1000/1000 [==============================] - 212s 212ms/step - loss: 1.0016 - accuracy: 0.7944 - val_loss: 0.9659 - val_accuracy: 0.8243
Epoch 6/10
1000/1000 [==============================] - 212s 212ms/step - loss: 0.8846 - accuracy: 0.8155 - val_loss: 0.8980 - val_accuracy: 0.8375
Epoch 7/10
1000/1000 [==============================] - 212s 212ms/step - loss: 0.8057 - accuracy: 0.8293 - val_

In [11]:
print("Saving model...")
model.save(model_save_path)
print("Saved")

Saving model...
INFO:tensorflow:Assets written to: /model/assets


Saved


In [12]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test set:\n Loss: {:0.3f}\n Accuracy: {:0.3f}".format(loss, accuracy))

625/625 [==============================] - 13s 21ms/step - loss: 0.7795 - accuracy: 0.8621
Test set:
 Loss: 0.780
 Accuracy: 0.862


In [ ]:
saved_model = tf.keras.models.load_model(model_save_path)

In [42]:
def predict(text , model):
    inp = [text]
    seq = tokenizer.texts_to_sequences(inp)
    padded = pad_sequences(seq , maxlen=MAX_SEQUENCE_LENGTH)
    prediction = labels[np.argmax(model.predict(padded))]
    print(prediction)


text= input("Your product name: ")

predict(text , model)
# predict(text , saved_model)

Your product name: ماوس بی سیم شیائومی مدل WXSMSBMW02
Mouse
